In [1]:
import sqlglot
from sqlglot import exp
import sqlglot.optimizer
from sqlglot.optimizer.eliminate_subqueries import eliminate_subqueries
from sqlglot.optimizer.scope import build_scope, find_all_in_scope
import copy
from typing import Dict, Optional, Tuple
import logging
logging.basicConfig(level=logging.WARN)

In [2]:
file = "./sql/old_join.sql"
with open(file, "r") as f:
    sql = f.read()

In [4]:
from oracle_glot.convert import oracle_to_ansi_transform, oracle_to_ansi_sql
print(oracle_to_ansi_sql(sql)) 


WITH table3 AS (
  SELECT
    tableInner1.id
  FROM tableInner1
  LEFT JOIN tableInner2
    ON tableInner1.id = tableInner2.id
)
SELECT
  table1.id,
  table2.cloumn1
FROM table1
RIGHT JOIN table2
  ON table2.id = table3.id AND table1.id = table2.id
LEFT JOIN table3
  ON table1.id = table3.id
WHERE
  table1.id = 1


In [4]:
simple_ast = sqlglot.parse_one("select * from t1 join t2 on t1.a = t2.a where a = 1", dialect="oracle")
replace_ast = sqlglot.parse_one("select t1.a from t1 join t2 on t1.a = t2.a", dialect="oracle")

node = simple_ast.find(exp.Select)
node.root().replace(replace_ast.root())
print(repr(simple_ast))

Select(
  expressions=[
    Star()],
  from=From(
    this=Table(
      this=Identifier(this=t1, quoted=False))),
  joins=[
    Join(
      this=Table(
        this=Identifier(this=t2, quoted=False)),
      on=EQ(
        this=Column(
          this=Identifier(this=a, quoted=False),
          table=Identifier(this=t1, quoted=False)),
        expression=Column(
          this=Identifier(this=a, quoted=False),
          table=Identifier(this=t2, quoted=False))))],
  where=Where(
    this=EQ(
      this=Column(
        this=Identifier(this=a, quoted=False)),
      expression=Literal(this=1, is_string=False))))
